In [1]:
print("Jai Shree Shyam!")

Jai Shree Shyam!


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load API key from environment variable - NEVER hardcode!
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Please set GOOGLE_API_KEY in your .env file")

In [11]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [12]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# !pip install google-adk

  Obtaining dependency information for google-adk from https://files.pythonhosted.org/packages/f2/ab/437f74e1357e1a6e0d1d7bdb8825744a6e178d0411e0d5e32664e9acdcd8/google_adk-1.19.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiosqlite>=0.21.0 from https://files.pythonhosted.org/packages/f5/10/6c25ed6de94c49f88a91fa5018cb4c0f3625f31d5be9f771ebe5cc7cd506/aiosqlite-0.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for authlib<2.0.0,>=1.5.1 from https://files.pythonhosted.org/packages/f8/aa/5082412d1ee302e9e7d80b6949bc4d2a8fa1149aaab610c5fc24709605d6/authlib-1.6.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for click<9.0.0,>=8.1.8 from https://files.pythonhosted.org/packages/98/78/01c019cdb5d6498122777c1a43056ebb3ebfeef2076d9d026bfe15583b2b/click-8.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi<0.119.0,>=0.115.0 from https://files.pythonhosted.org/packages/24/04/2f9e8a965f4214883258a6f716fea324d1b81e97

In [13]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [14]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        # retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [15]:
# runner = InMemoryRunner(agent=root_agent)
#
# print("✅ Runner created.")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


✅ Runner created.


In [19]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations. You have to gather and filter live data from the internet including events, social buzz and activity listings to aggregate relevant suggestoins for the users""",
    tools=[google_search],
    output_key="info_miner_web_data_aggregator",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created!")

✅ research_agent created!


In [20]:
# response = await runner.run_debug(
#     "What is graph api from Google? What languages is the SDK available in?"
# )

In [22]:
# response = await runner.run_debug("What's the weather in bengaluru today?")

In [28]:
from google.adk.tools import AgentTool

# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
itinerary_agent = Agent(
    name="ItineraryCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a travel guide that acts a travel guide (for one day) in Bengaluru that gives itinerary time wise. You can use google maps data and social media data to give response. The information you receive includes fields -
type: This is the type of plan eg. full day, evening, morning, night etc. (Compulsory)
people: The type of crowd like family, friends, couple, corporate etc. (Compulsory)
numberOfPeople: The size of people (Compulsory)
occasion: why the core reason the plan needs to be made (Optional)
inclusions: what things are needed in the plan like lunch, breakfast, temple, zoo, nature, dinner, gaming, music, museums, tourist attractions (Optional)
budget: The total amount that needed for this including all people (Optional)
budgetMode: with or without traffic (Optional) (by default consider with)
transportMode: auto, cab, bus, metro etc. (Optional) (For budget mode without traffic don't consider this)
location: start location for the plan (Compulsory)
dateOfPlan: execution date of the plan (Compulsory)
startTime: the start time for the plan (Compulsory)
remarks: Other remarks by the user (Optional)


The research_agent tool can be used for internet information to give the final response correct.
Use available tools to enrich results (geocode/distance/research).

As provided you will be given all Compulsory fields and some or none optional fields.
Output Format
List of [Time of Day, Activity/Travel ,Total Spendings]

Consider events happening on that date, traffic, weather and also give restaurants name when asked. Consider user remarks when provided""",
    # We wrap the sub-agents in ⁠ AgentTool ⁠ to make them callable tools for the root agent.
    tools=[AgentTool(research_agent)],  #, AgentTool(distance_agent), AgentTool(geocode_agent)
    output_key="travel_itinerary",  # The final itinerary will be stored in the session
)

print("✅ Itinerary agent created.")

✅ Itinerary agent created.


In [29]:
runner = InMemoryRunner(agent=itinerary_agent)

print("✅ Runner created.")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


✅ Runner created.


In [30]:
response = await runner.run_debug(
)


 ### Created new session: debug_session_id

User > type:fullday, people:friends, numberOfPeople:8, occasion:weekend fun, inclusions:temple, zoo, nature, location:nift college, hsr layout, dateOfPlan:5 December 2025, startTime:10 AM morning


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/suyoggupta/PycharmProjects/Cardano/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".


ItineraryCoordinator > Here's a plan for your weekend fun with friends in Bengaluru:

**Full Day Itinerary: Friends' Weekend Fun in Bengaluru**

*   **Type:** Full Day
*   **People:** Friends
*   **Number of People:** 8
*   **Occasion:** Weekend Fun
*   **Inclusions:** Temple, Zoo, Nature
*   **Location:** NIFT College, HSR Layout
*   **Date:** December 5th, 2025
*   **Start Time:** 10:00 AM

**Itinerary:**

*   **10:00 AM - 11:30 AM:** Start your day with a visit to the **Ragigudda Sri Prasanna Anjaneyaswamy Temple**. It's a well-known temple that offers a peaceful start to your day.
    *   *Estimated Travel Time from NIFT College:* ~15-20 minutes
*   **11:30 AM - 1:30 PM:** Head to **Lalbagh Botanical Garden**. Explore the diverse flora, enjoy the serene atmosphere, and take a relaxing walk.
    *   *Estimated Travel Time from Ragigudda Temple:* ~20-25 minutes
*   **1:30 PM - 2:30 PM:** Lunch. You can find many dining options around Lalbagh or head back towards HSR Layout. For a goo